In [1]:
# pip install --upgrade nbformat

# Imports
## NOTE:
You will need to have installed the Google Cloud SDK and a project set up to run this work flow interactively. Instructions on how to set up the Google Cloud SDK are [here](https://cloud.google.com/sdk/docs/install).

In [2]:
import ee

from gee_water.utils import (
    add_ndvi_l30,
    create_monthly_composites,
    get_time_series,
    get_monthly_image
)
from gee_water.visualization_utils import plot_multiple_timeseries_with_trendlines_plotly, create_split_viirs_map
from gee_water.masks import mask_hls_l30_cloud_shadow
from gee_water.vis_params import ndvi_vis_params


In [3]:
ee.Initialize()

# README
The purpose of this notebook is to do some exploratory work on NDVI for regions in Southern Baja for a blog post exploring water resources in the area.
The focus region for this project will be Baja California Sur, near the southern most tip of the peninsula in the La Paz - Cabo - Todos Santos triangle.

## Define Region of Interest

In [4]:
# Coordinates for La Paz, BCS
la_paz_coords = ee.Geometry.Point([-110.3166667, 24.1422222])
buffer_distance_meters = 75 * 1609.34  # 75 miles in meters
roi = la_paz_coords.buffer(buffer_distance_meters)

# Run NDVI Analysis Using HLS GEE Data Set

In [5]:
# For this particular blog, I am analyzing multiple datasets from a handful of different 
# satellites. The year 2013 is far enough back historically to capture form trends, but still 
# allows for higih quality data to be gathred for all my metrics of interest.
start_date = '2013-01-01'
end_date   = '2023-12-31'


In [6]:
hls_l30_with_ndvi = (
    ee.ImageCollection('NASA/HLS/HLSL30/v002')
    .filterDate(start_date, end_date)
    .filterBounds(roi)
    .map(mask_hls_l30_cloud_shadow)
    .map(add_ndvi_l30)
)


In [7]:
# create composites
hls_l30_ndvi_monthly = create_monthly_composites(hls_l30_with_ndvi)

In [8]:
# create time series data from the monthly composites
hls_l30_ndvi_timeseries = get_time_series(hls_l30_ndvi_monthly, 'NDVI', roi)

In [9]:
plot_multiple_timeseries_with_trendlines_plotly(
    df_list=[hls_l30_ndvi_timeseries],
    date_col='date',
    value_col='NDVI',
    labels=['HLS NDVI'],
    x_interval_months=6,
    title='HLS Derived NDVI 2013 - 2023',
    ylabel='Value (units)'
)

# Determine if Trend is Statistically Significant

# Display NDVI Year to Year

In [11]:
# June early NDVI
hls_ndvi_june_2013 = get_monthly_image(hls_l30_with_ndvi, 'NDVI', 2013, 6)

# June 2023 NDVI
hls_ndvi_june_2023 = get_monthly_image(hls_l30_with_ndvi, 'NDVI', 2023, 6)

# December early NDVI
hls_ndvi_dec_2013 = get_monthly_image(hls_l30_with_ndvi, 'NDVI', 2013, 12)

# December 2023 NDVI
hls_ndvi_dec_2023 = get_monthly_image(hls_l30_with_ndvi, 'NDVI', 2023, 12)


In [15]:
create_split_viirs_map(
        left_image=hls_ndvi_june_2013, 
        right_image=hls_ndvi_june_2023,
        left_image_label="HLS NDVI June 2013",
        right_image_label="HLS NDVI June 2023",
        vis_params=ndvi_vis_params,
        mep_center_coords=[24.1422222, -110.3166667],
        zoom=10,
        legend_title='NDVI',
        legend_labels= ['Low (0)', 'Mid (0.5)', 'High (1)'],
        html_export_location='./../static/html/ndvi_test_2.html'
)

Map(center=[24.1422222, -110.3166667], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_tit…